<a href="https://colab.research.google.com/github/MdNiamul/DataHioding/blob/main/Modern_%F0%9F%94%90_Bengali_Steganography_Suite_AES_256_GCM_%2B_zlib_Compression_%E2%80%A2_Quantum_Ready_Security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from google.colab import files
import io
from PIL import Image
import base64
import hashlib
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import secrets, os, json, time
import zlib
from datetime import datetime, timezone

# ---------- Enhanced AES-256-GCM Encryption Class with zlib Compression ----------
class SecureEncryption:
    """Enhanced AES-256-GCM encryption with proper security practices and zlib compression"""

    @staticmethod
    def derive_key_256(password: str, salt: bytes, iterations: int = 100000) -> bytes:
        """Derive a 256-bit key using PBKDF2 with SHA-256"""
        if len(salt) != 16:
            raise ValueError("Salt must be exactly 16 bytes")

        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # 256 bits = 32 bytes
            salt=salt,
            iterations=iterations,
            backend=default_backend()
        )
        return kdf.derive(password.encode('utf-8'))

    @staticmethod
    def encrypt_aes256_gcm(plaintext: bytes, password: str) -> bytes:
        """
        Encrypt data using AES-256-GCM with secure random salt and nonce
        Now includes optional zlib compression for space efficiency
        Returns: JSON-encoded bytes containing salt, nonce, ciphertext, and compression flag
        """
        try:
            # --- 1. Optional zlib compression ---
            comp = zlib.compress(plaintext, level=9)
            compressed = len(comp) < len(plaintext)           # keep only if it saves space
            payload_bytes = comp if compressed else plaintext

            # --- 2. AES-256-GCM exactly as before ---
            salt = secrets.token_bytes(16)  # 128-bit salt
            nonce = secrets.token_bytes(12)  # 96-bit nonce (recommended for GCM)
            key = SecureEncryption.derive_key_256(password, salt, 100_000)
            aesgcm = AESGCM(key)
            ciphertext = aesgcm.encrypt(nonce, payload_bytes, None)

            # --- 3. JSON envelope ---
            encrypted_package = {
                'version': '2.2',  # Updated version for compression support
                'algorithm': 'AES-256-GCM',
                'iterations': 100_000,
                'salt': base64.b64encode(salt).decode('ascii'),
                'nonce': base64.b64encode(nonce).decode('ascii'),
                'ciphertext': base64.b64encode(ciphertext).decode('ascii'),
                'compressed': compressed            # <-- NEW flag
            }
            return json.dumps(encrypted_package, separators=(',', ':')).encode('utf-8')

        except Exception as e:
            raise ValueError(f"AES-256-GCM encryption failed: {str(e)}")

    @staticmethod
    def decrypt_aes256_gcm(encrypted_data: bytes, password: str) -> bytes:
        """
        Decrypt AES-256-GCM encrypted data with zlib decompression support
        Returns: Original plaintext bytes
        """
        try:
            # Parse encrypted package
            package = json.loads(encrypted_data.decode('utf-8'))

            # Verify format
            if package.get('algorithm') != 'AES-256-GCM':
                raise ValueError(f"Unsupported algorithm: {package.get('algorithm')}")

            # Extract components
            salt = base64.b64decode(package['salt'])
            nonce = base64.b64decode(package['nonce'])
            ciphertext = base64.b64decode(package['ciphertext'])
            iterations = package.get('iterations', 100000)

            # Derive key with same parameters
            key = SecureEncryption.derive_key_256(password, salt, iterations)

            # Initialize AES-256-GCM
            aesgcm = AESGCM(key)

            # Decrypt and verify authentication
            plaintext = aesgcm.decrypt(nonce, ciphertext, None)

            # NEW: Handle decompression if needed
            if package.get('compressed', False):
                plaintext = zlib.decompress(plaintext)

            return plaintext

        except json.JSONDecodeError:
            raise ValueError("Invalid encrypted data format")
        except Exception as e:
            raise ValueError(f"AES-256-GCM decryption failed: {str(e)}")

    @staticmethod
    def estimate_encrypted_size(plaintext_size: int) -> int:
        """Accurately estimate encrypted data size with optional compression"""
        # Test compression on a sample to get realistic estimate
        sample_data = b'A' * min(plaintext_size, 1000)  # Sample for compression test
        compressed_sample = zlib.compress(sample_data, level=9)
        compression_ratio = len(compressed_sample) / len(sample_data)

        # Use compression ratio if beneficial
        if compression_ratio < 1.0:
            effective_size = int(plaintext_size * compression_ratio)
        else:
            effective_size = plaintext_size

        ciphertext_size = effective_size + 16  # Add GCM auth tag
        ciphertext_b64_size = ((ciphertext_size + 2) // 3) * 4  # Base64 encoding

        # JSON structure with all fields (including compression flag)
        json_overhead = 220  # Slightly more for compression flag
        salt_b64 = 24  # 16 bytes -> 24 base64 chars
        nonce_b64 = 16  # 12 bytes -> 16 base64 chars

        total_size = json_overhead + salt_b64 + nonce_b64 + ciphertext_b64_size

        return total_size

# ---------- Enhanced Shuffle Engine ----------
class DynamicShuffle:
    def __init__(self, seed=None):
        self.seed = seed or int(time.time() * 1000)

    def miller_shuffle_lite(self, index, shuffle_id, limit):
        """Consistent shuffling algorithm for encode/decode parity"""
        if limit <= 1:
            return 0
        x = (shuffle_id ^ index) & 0xFFFFFFFF
        x = ((x >> 16) ^ x) * 0x45d9f3b
        x = ((x >> 16) ^ x) * 0x45d9f3b
        x = (x >> 16) ^ x
        return x % limit

    def generate_shuffle_sequence(self, length, shuffle_id):
        """Generate consistent shuffle sequence using Fisher-Yates"""
        if length <= 0:
            return []

        # Create initial sequence
        seq = list(range(length))

        # Fisher-Yates shuffle with deterministic random
        for i in range(length - 1, 0, -1):
            j = self.miller_shuffle_lite(i, shuffle_id, i + 1)
            seq[i], seq[j] = seq[j], seq[i]

        return seq

# ---------- Enhanced Bengali Steganography Class ----------
class BengaliSteganographyAES256:
    def __init__(self):
        self.original_image = None
        self.decode_image = None
        self.shuffle_engine = DynamicShuffle()
        self.encryption = SecureEncryption()
        self.dark_mode = False
        self.max_capacity_bytes = 0
        self.workflow_step = 1  # Track workflow progression
        self.encoded_result = None  # Store encoded image for download
        self.create_widgets()
        self.setup_event_handlers()

    # ---------- UI Setup ----------
    def create_widgets(self):
        colors = self.get_theme_colors()

        # Modern Dark mode toggle with glass effect
        self.dark_mode_toggle = widgets.ToggleButton(
            value=False,
            description='🌙 Dark Mode',
            button_style='',
            style={'button_color': 'transparent', 'font_weight': 'bold'},
            layout={'width': '160px', 'height': '45px'}
        )
        self.dark_mode_toggle.observe(self.toggle_dark_mode, names='value')

        # Add custom CSS for the toggle button
        self.apply_modern_button_style(self.dark_mode_toggle, is_toggle=True)

        # Main tabs with modern styling
        self.tab = widgets.Tab()
        self.encode_tab = widgets.VBox()
        self.decode_tab = widgets.VBox()
        self.tab.children = [self.encode_tab, self.decode_tab]
        self.tab.set_title(0, '🔐 Encode Message')
        self.tab.set_title(1, '🔓 Decode Message')

        # Enhanced Encode UI with workflow steps
        self.workflow_display = widgets.HTML()

        # Step 1: Modern Image Upload Button
        self.encode_upload = widgets.FileUpload(
            description='📤 Step 1: Upload Image',
            accept='image/*',
            multiple=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='60px')
        )
        self.apply_modern_upload_style(self.encode_upload)

        self.encode_image_display = widgets.Output()
        self.capacity_display = widgets.HTML()

        # Step 2: AES-256 Password Setup with modern design
        self.password_section = widgets.VBox([
            widgets.HTML('<h4 style="padding:15px; border-radius:12px; background:linear-gradient(135deg, #6366f1 0%, #8b5cf6 50%, #ec4899 100%); color:white; text-shadow:0 2px 4px rgba(0,0,0,0.3); box-shadow:0 8px 32px rgba(99, 102, 241, 0.3);">🔑 Step 2: AES-256 Password Setup</h4>'),
            widgets.HTML('''
            <div style="background:linear-gradient(135deg, rgba(30, 41, 59, 0.95) 0%, rgba(51, 65, 85, 0.9) 100%);
                        border:2px solid rgba(99, 102, 241, 0.4); padding:20px; border-radius:16px;
                        margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.15); backdrop-filter:blur(10px);">
                <div style="color:white; font-weight:bold; margin-bottom:12px; font-size:18px; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                    🔐 AES-256-GCM Security with zlib Compression:
                </div>
                <ul style="color:#e2e8f0; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                    <li style="margin:8px 0;">• <span style="color:#60a5fa;">256-bit encryption key</span> with quantum-resistant design</li>
                    <li style="margin:8px 0;">• <span style="color:#34d399;">PBKDF2 with 100,000 iterations</span> for enhanced security</li>
                    <li style="margin:8px 0;">• <span style="color:#fbbf24;">Cryptographically secure</span> salt & nonce generation</li>
                    <li style="margin:8px 0;">• <span style="color:#f87171;">Built-in authentication</span> verification</li>
                    <li style="margin:8px 0;">• <span style="color:#a78bfa; font-weight:bold;">NEW:</span> <span style="color:#10b981;">Automatic zlib compression</span> (level 9)</li>
                </ul>
            </div>
            ''')
        ])

        # Modern password inputs with glass morphism
        self.password_input = widgets.Password(
            placeholder='Enter AES-256 password (min 12 chars recommended)',
            layout={'width': '450px', 'height': '50px'},
            style={'description_width': 'initial'},
            disabled=True
        )
        self.apply_modern_input_style(self.password_input)

        self.password_confirm = widgets.Password(
            placeholder='Confirm AES-256 password',
            layout={'width': '450px', 'height': '50px'},
            style={'description_width': 'initial'},
            disabled=True
        )
        self.apply_modern_input_style(self.password_confirm)

        self.password_strength = widgets.HTML()

        # Step 3: Modern Security Options
        self.security_section = widgets.VBox([
            widgets.HTML('<h4 style="color:#e2e8f0; background:linear-gradient(135deg, #1e293b 0%, #334155 100%); padding:15px; border-radius:12px; box-shadow:0 8px 32px rgba(0,0,0,0.2);">🛡️ Step 3: Security & Compression Options</h4>'),
            widgets.HTML('''
            <div style="background:linear-gradient(135deg, rgba(15, 23, 42, 0.95) 0%, rgba(30, 41, 59, 0.9) 100%);
                        border:2px solid rgba(34, 197, 94, 0.4); padding:20px; border-radius:16px;
                        margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.15); backdrop-filter:blur(10px);">
                <div style="color:#22c55e; font-weight:bold; margin-bottom:12px; font-size:18px; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                    💡 Enhanced Security & Compression Features:
                </div>
                <ul style="color:#e2e8f0; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                    <li style="margin:8px 0;"><strong style="color:#fb7185;">SHA-256 Hash:</strong> <span style="color:#94a3b8;">Verifies message integrity during decoding</span></li>
                    <li style="margin:8px 0;"><strong style="color:#fb7185;">Timestamp:</strong> <span style="color:#94a3b8;">Records when the message was encoded</span></li>
                    <li style="margin:8px 0;"><strong style="color:#67e8f9;">zlib Compression:</strong> <span style="color:#94a3b8;">Automatically reduces data size when beneficial</span></li>
                    <li style="margin:8px 0;"><span style="color:#a78bfa;">All features add minimal overhead but significantly improve efficiency</span></li>
                </ul>
            </div>
            ''')
        ])

        # Modern checkboxes
        self.add_message_hash = widgets.Checkbox(
            description='✅ Add SHA-256 Message Hash (Strongly Recommended)',
            value=True,
            disabled=True,
            style={'description_width': 'initial'}
        )

        self.add_timestamp = widgets.Checkbox(
            description='🕒 Add Timestamp',
            value=True,
            disabled=True,
            style={'description_width': 'initial'}
        )

        # Step 4: Message Input with modern styling
        self.message_section = widgets.VBox([
            widgets.HTML('<h4 style="color:#e2e8f0; background:linear-gradient(135deg, #7c3aed 0%, #a855f7 100%); padding:15px; border-radius:12px; box-shadow:0 8px 32px rgba(124, 58, 237, 0.3);">📝 Step 4: Message Input</h4>')
        ])

        self.message_input = widgets.Textarea(
            placeholder='আপনার বার্তা লিখুন... (Write your message here)',
            layout={'width': '100%', 'height': '140px'},
            disabled=True,
            style={'description_width': 'initial'}
        )
        self.apply_modern_input_style(self.message_input, is_textarea=True)

        self.message_stats = widgets.HTML()

        # Step 5: Modern Encode section
        self.encode_section = widgets.VBox([
            widgets.HTML('<h4 style="color:white; background:linear-gradient(135deg, #059669 0%, #10b981 100%); padding:15px; border-radius:12px; box-shadow:0 8px 32px rgba(5, 150, 105, 0.4);">🚀 Step 5: Encode Message</h4>')
        ])

        # Modern encode button with amber gradient
        self.encode_button = widgets.Button(
            description='🔐 Encode with AES-256-GCM + zlib',
            button_style='',
            layout={'width': '320px', 'height': '55px'},
            disabled=True
        )
        self.apply_modern_button_style(self.encode_button, 'success')

        # Modern download button with amber gradient
        self.download_button = widgets.Button(
            description='📥 Download Encoded Image',
            button_style='',
            layout={'width': '280px', 'height': '55px'}
        )
        self.apply_modern_button_style(self.download_button, 'info')
        self.download_button.layout.display = 'none'  # Hide initially

        self.encode_output = widgets.Output()

        # Enhanced Decode UI with modern styling
        self.decode_upload = widgets.FileUpload(
            description='📤 Upload Encoded Image',
            accept='image/*',
            multiple=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='60px')
        )
        self.apply_modern_upload_style(self.decode_upload)

        self.decode_image_display = widgets.Output()

        self.decode_password = widgets.Password(
            placeholder='Enter AES-256 password used during encoding',
            layout={'width': '450px', 'height': '50px'},
            style={'font_weight': 'bold'}
        )
        self.apply_modern_input_style(self.decode_password)

        self.decode_button = widgets.Button(
            description='🔓 Decode with AES-256-GCM + zlib',
            button_style='',
            layout={'width': '320px', 'height': '55px'}
        )
        self.apply_modern_button_style(self.decode_button, 'warning')

        self.decode_output = widgets.Output()

        # Layout assembly
        password_box = widgets.HBox([self.password_input, self.password_confirm], layout={'align_items': 'center', 'justify_content': 'space-around'})
        security_box = widgets.VBox([self.add_message_hash, self.add_timestamp])
        action_buttons = widgets.HBox([self.encode_button, self.download_button], layout={'align_items': 'center', 'justify_content': 'center', 'margin': '20px 0'})

        self.encode_tab.children = [
            self.workflow_display,
            self.encode_upload,
            self.encode_image_display,
            self.capacity_display,
            self.password_section,
            password_box,
            self.password_strength,
            self.security_section,
            security_box,
            self.message_section,
            self.message_input,
            self.message_stats,
            self.encode_section,
            action_buttons,
            self.encode_output
        ]

        self.decode_tab.children = [
            widgets.HTML('<h3 style="color:white; background:linear-gradient(135deg, #0ea5e9 0%, #3b82f6 100%); padding:20px; border-radius:12px; text-align:center; box-shadow:0 8px 32px rgba(14, 165, 233, 0.3);">🔓 AES-256-GCM + zlib Decoding</h3>'),
            self.decode_upload,
            self.decode_image_display,
            widgets.HBox([self.decode_password], layout={'justify_content': 'center', 'margin': '20px 0'}),
            widgets.HBox([self.decode_button], layout={'justify_content': 'center', 'margin': '20px 0'}),
            self.decode_output
        ]

        # Ultra-modern main title with animated gradient
        title_html = '''
        <div style="text-align:center;
                    background:linear-gradient(135deg, #667eea 0%, #764ba2 25%, #f093fb 50%, #f5576c 75%, #4facfe 100%);
                    background-size:400% 400%;
                    animation:gradientShift 8s ease infinite;
                    padding:30px; border-radius:20px; margin:25px 0;
                    box-shadow:0 20px 60px rgba(0,0,0,0.3), 0 8px 32px rgba(118, 75, 162, 0.4);
                    position:relative; overflow:hidden;">
            <div style="position:absolute; top:0; left:0; right:0; bottom:0;
                        background:rgba(255,255,255,0.1); backdrop-filter:blur(10px);"></div>
            <h1 style="margin:0; color:white; font-size:32px; font-weight:900;
                       text-shadow:0 4px 8px rgba(0,0,0,0.5); letter-spacing:2px;
                       position:relative; z-index:1;">
                🔐 Bengali Steganography Suite
            </h1>
            <p style="margin:15px 0 0 0; color:rgba(255,255,255,0.9); font-size:18px;
                      text-shadow:0 2px 4px rgba(0,0,0,0.3); position:relative; z-index:1; font-weight:600;">
                AES-256-GCM + zlib Compression • Quantum-Ready Security
            </p>
        </div>
        <style>
        @keyframes gradientShift {
            0% { background-position: 0% 50%; }
            50% { background-position: 100% 50%; }
            100% { background-position: 0% 50%; }
        }
        </style>
        '''

        self.main_container = widgets.VBox([
            widgets.HTML(title_html),
            widgets.HBox([
                widgets.HTML('<div style="flex-grow:1;"></div>'),  # Spacer
                self.dark_mode_toggle
            ], layout={'justify_content': 'flex-end', 'margin': '10px 0'}),
            self.tab
        ])

        self.update_workflow_display()

    def get_theme_colors(self):
        """
        Return a vibrant, modern color-palette that switches between
        a cyber-neon (dark) and a clean-modern (light) theme.
        """
        if self.dark_mode:                       # CYBER-NEON - IMPROVED CONTRAST
            return {
                'bg_primary': '#0a0a0a',      # Darker background for better contrast
                'bg_secondary': '#1a1a1a',    # Slightly lighter for depth
                'text_primary': '#ffffff',    # Pure white for maximum readability
                'text_secondary': '#e2e8f0',  # Light gray for secondary text
                'success': '#00ff88',         # Electric green (kept)
                'error': '#ff3333',           # Brighter red for visibility
                'info': '#4fc3f7',            # Brighter blue for dark mode
                'warning': '#ffb74d',         # Brighter orange for visibility
                'border': '#444444',          # Lighter border for definition
                'card_bg': 'rgba(30, 30, 30, 0.9)'  # Lighter card background
            }
        else:                                    # CLEAN-MODERN (unchanged)
            return {
                'bg_primary': '#ffffff',
                'bg_secondary': '#f8fafc',
                'text_primary': '#1e293b',
                'text_secondary': '#64748b',
                'success': '#10b981',
                'error': '#ef4444',
                'info': '#3b82f6',
                'warning': '#f59e0b',
                'border': '#e2e8f0',
                'card_bg': 'rgba(255, 255, 255, 0.9)'
            }

    def apply_modern_upload_style(self, upload_widget):
        """Apply modern electric green styling for upload buttons"""
        custom_css = '''
        <style>
        .widget-upload {
            background: linear-gradient(135deg, #00ff88 0%, #00d4ff 100%) !important;
            border: 2px solid rgba(0, 255, 136, 0.5) !important;
            border-radius: 15px !important;
            box-shadow: 0 10px 25px rgba(0, 255, 136, 0.4), 0 4px 10px rgba(0, 0, 0, 0.2) !important;
            color: white !important;
            font-weight: bold !important;
            text-shadow: 0 2px 4px rgba(0, 0, 0, 0.5) !important;
            backdrop-filter: blur(10px) !important;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
            position: relative !important;
            overflow: hidden !important;
        }

        .widget-upload:hover {
            transform: translateY(-3px) !important;
            box-shadow: 0 15px 35px rgba(0, 255, 136, 0.6), 0 8px 15px rgba(0, 0, 0, 0.3) !important;
            border-color: rgba(0, 255, 136, 0.8) !important;
        }

        .widget-upload .widget-upload-input {
            color: white !important;
            font-weight: bold !important;
        }

        .widget-upload-label {
            color: white !important;
            font-weight: bold !important;
            text-shadow: 0 2px 4px rgba(0, 0, 0, 0.5) !important;
        }
        </style>
        '''
        display(HTML(custom_css))

    def apply_modern_button_style(self, button, style_type='primary', is_toggle=False):
        """Apply modern button styling with amber color for main action buttons"""

        # Use amber color for main action buttons (encode, download, decode)
        if style_type in ['success', 'info', 'warning']:
            # Amber gradient for all main action buttons
            bg_gradient = "linear-gradient(135deg, #f59e0b 0%, #fbbf24 100%)"
            shadow_color = "rgba(245, 158, 11, 0.6)"
            hover_shadow = "0 15px 35px rgba(245, 158, 11, 0.8)"
            border_color = "rgba(245, 158, 11, 0.4)"
        else:
            # Default style for other buttons
            if self.dark_mode:
                bg_gradient = "linear-gradient(135deg, #2d3748 0%, #4a5568 100%)"
                shadow_color = "rgba(45, 55, 72, 0.6)"
                hover_shadow = "0 15px 35px rgba(45, 55, 72, 0.8)"
                border_color = "rgba(255, 255, 255, 0.3)"
            else:
                bg_gradient = "linear-gradient(135deg, #4b5563 0%, #1f2937 100%)"
                shadow_color = "rgba(31, 41, 55, 0.4)"
                hover_shadow = "0 15px 35px rgba(31, 41, 55, 0.6)"
                border_color = "rgba(255, 255, 255, 0.3)"

        # Set the button style
        button.style.button_color = 'transparent'
        button.style.font_weight = 'bold'
        button.style.text_color = 'white'

        # Create custom CSS
        custom_css = f"""
        <style>
        .custom-modern-button {{
            background: {bg_gradient} !important;
            border: 2px solid {border_color} !important;
            border-radius: 15px !important;
            box-shadow: 0 10px 25px {shadow_color}, 0 4px 10px rgba(0, 0, 0, 0.2) !important;
            color: white !important;
            font-weight: bold !important;
            text-shadow: 0 2px 4px rgba(0, 0, 0, 0.5) !important;
            backdrop-filter: blur(10px) !important;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
            position: relative !important;
            overflow: hidden !important;
        }}

        .custom-modern-button:hover {{
            transform: translateY(-3px) !important;
            box-shadow: {hover_shadow}, 0 8px 15px rgba(0, 0, 0, 0.3) !important;
            border-color: rgba(255, 255, 255, 0.5) !important;
        }}

        .custom-modern-button:active {{
            transform: translateY(-1px) !important;
            box-shadow: 0 5px 15px {shadow_color} !important;
        }}

        .custom-modern-button::before {{
            content: '' !important;
            position: absolute !important;
            top: 0 !important;
            left: -100% !important;
            width: 100% !important;
            height: 100% !important;
            background: linear-gradient(90deg, transparent, rgba(255, 255, 255, 0.3), transparent) !important;
            transition: left 0.5s !important;
        }}

        .custom-modern-button:hover::before {{
            left: 100% !important;
        }}
        </style>
        """

        # Display the CSS
        display(HTML(custom_css))

    def apply_modern_input_style(self, input_widget, is_textarea=False):
        """Apply modern glassmorphism input styling"""

        height = '140px' if is_textarea else '50px'

        custom_css = f"""
        <style>
        .custom-modern-input {{
            background: rgba(255, 255, 255, 0.1) !important;
            border: 2px solid rgba(99, 102, 241, 0.3) !important;
            border-radius: 12px !important;
            color: #374151 !important;
            backdrop-filter: blur(10px) !important;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1) !important;
            transition: all 0.3s ease !important;
            font-size: 14px !important;
            padding: 12px 16px !important;
        }}

        .custom-modern-input:focus {{
            border-color: rgba(99, 102, 241, 0.6) !important;
            box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1), 0 12px 40px rgba(0, 0, 0, 0.15) !important;
            transform: translateY(-2px) !important;
        }}

        .custom-modern-input::placeholder {{
            color: rgba(107, 114, 128, 0.7) !important;
        }}
        </style>
        """

        display(HTML(custom_css))

    def print_boxed_messages(self, messages, msg_type="info"):
        """Print messages in styled boxes with FIXED dark mode contrast"""
        colors = self.get_theme_colors()

        # FIXED: Use dark gray background for all message types in dark mode with better contrast
        if self.dark_mode:
            type_config = {
                "success": {"color": "#00ff88", "icon": "✅", "bg": "rgba(55, 65, 81, 0.9)", "text_color": "#e5e7eb"},  # Dark gray background with light text
                "error": {"color": "#ff3333", "icon": "❌", "bg": "rgba(55, 65, 81, 0.9)", "text_color": "#e5e7eb"},
                "warning": {"color": "#ffb74d", "icon": "⚠️", "bg": "rgba(55, 65, 81, 0.9)", "text_color": "#e5e7eb"},
                "info": {"color": "#4fc3f7", "icon": "ℹ️", "bg": "rgba(55, 65, 81, 0.9)", "text_color": "#e5e7eb"}
            }
        else:
            type_config = {
                "success": {"color": "#10b981", "icon": "✅", "bg": "rgba(16, 185, 129, 0.1)", "text_color": "#1e293b"},
                "error": {"color": "#ef4444", "icon": "❌", "bg": "rgba(239, 68, 68, 0.1)", "text_color": "#1e293b"},
                "warning": {"color": "#f59e0b", "icon": "⚠️", "bg": "rgba(245, 158, 11, 0.1)", "text_color": "#1e293b"},
                "info": {"color": "#3b82f6", "icon": "ℹ️", "bg": "rgba(59, 130, 246, 0.1)", "text_color": "#1e293b"}
            }

        config = type_config.get(msg_type, type_config["info"])

        # Create the box content with FIXED contrast
        box_content = []
        for message in messages:
            # Extract icon if present in message
            if message.startswith(("✅", "❌", "⚠️", "ℹ️", "🔐", "🕒", "📊", "🗜️", "🔀", "📷", "🔓")):
                icon = message[0]
                text = message[1:].strip()
            else:
                icon = config['icon']
                text = message

            box_content.append(f'''
            <div style="display:flex; align-items:center; padding:12px 0; border-bottom:1px solid {config['color']}30;">
                <span style="color:{config['color']}; font-size:18px; margin-right:12px; min-width:24px; font-weight:bold;">{icon}</span>
                <span style="flex:1; color:{config['text_color']}; font-weight:500;">{text}</span>
            </div>
            ''')

        message_html = f'''
        <div style="background:{config['bg']}; border:2px solid {config['color']}60;
                    padding:15px 20px; border-radius:12px; margin:10px 0;
                    box-shadow:0 6px 20px rgba(0,0,0,0.3); backdrop-filter:blur(10px);
                    border-left:4px solid {config['color']};">
            {''.join(box_content)}
        </div>
        '''
        display(HTML(message_html))

    def setup_event_handlers(self):
        """Setup all event handlers"""
        self.encode_button.on_click(self.encode_message)
        self.decode_button.on_click(self.decode_message)
        self.download_button.on_click(self.download_encoded_image)
        self.encode_upload.observe(self.handle_encode_upload, names='value')
        self.decode_upload.observe(self.handle_decode_upload, names='value')
        self.password_input.observe(self.validate_password, names='value')
        self.password_confirm.observe(self.validate_password, names='value')
        self.message_input.observe(self.update_message_stats, names='value')

    def update_workflow_display(self):
        """Update the workflow progress display with modern design"""
        colors = self.get_theme_colors()
        steps = [
            ('📤', 'Upload Image', self.workflow_step >= 2),
            ('🔑', 'Setup AES-256 Password', self.workflow_step >= 3),
            ('🛡️', 'Security Options', self.workflow_step >= 4),
            ('📝', 'Enter Message', self.workflow_step >= 5),
            ('🚀', 'Encode', self.workflow_step >= 6)
        ]

        step_html = []
        for i, (icon, title, completed) in enumerate(steps, 1):
            if completed:
                status_color = "linear-gradient(135deg, #10b981 0%, #34d399 100%)"
                border_color = "#10b981"
                status_icon = '✅'
                text_color = "white"
                shadow = "0 8px 25px rgba(16, 185, 129, 0.3)"
            elif i == self.workflow_step:
                status_color = "linear-gradient(135deg, #3b82f6 0%, #60a5fa 100%)"
                border_color = "#3b82f6"
                status_icon = '⏳'
                text_color = "white"
                shadow = "0 8px 25px rgba(59, 130, 246, 0.3)"
            else:
                status_color = "rgba(107, 114, 128, 0.2)"
                border_color = "#6b7280"
                status_icon = '⭕'
                text_color = "#9ca3af"
                shadow = "0 4px 10px rgba(0, 0, 0, 0.1)"

            step_html.append(f'''
                <div style="display:inline-block; margin:8px 12px; padding:12px 18px;
                           background:{status_color}; border:2px solid {border_color};
                           border-radius:25px; color:{text_color}; font-weight:bold;
                           box-shadow:{shadow}; backdrop-filter:blur(10px);
                           transition:all 0.3s ease; text-shadow:0 2px 4px rgba(0,0,0,0.2);">
                    {icon} {status_icon} {title}
                </div>
            ''')

        self.workflow_display.value = f'''
        <div style="text-align:center; padding:25px;
                    background:linear-gradient(135deg, rgba(15, 23, 42, 0.9) 0%, rgba(30, 41, 59, 0.8) 100%);
                    border:2px solid rgba(99, 102, 241, 0.3); border-radius:20px; margin:20px 0;
                    box-shadow:0 20px 40px rgba(0,0,0,0.15); backdrop-filter:blur(15px);">
            <h4 style="margin:0 0 20px 0; color:#60a5fa; font-size:20px; font-weight:bold;
                       text-shadow:0 2px 4px rgba(0,0,0,0.3);">📋 Encoding Workflow Progress</h4>
            <div style="display:flex; flex-wrap:wrap; justify-content:center; align-items:center;">
                {''.join(step_html)}
            </div>
        </div>
        '''

    def toggle_dark_mode(self, change):
        self.dark_mode = change['new']
        self.update_all_widgets()
        self.update_theme_dependent_sections()

    def update_theme_dependent_sections(self):
        """Update sections that need theme-specific styling"""
        if self.dark_mode:
            # Update password section for dark mode
            self.password_section.children = (
                widgets.HTML('<h4 style="padding:15px; border-radius:12px; background:linear-gradient(135deg, #6366f1 0%, #8b5cf6 50%, #ec4899 100%); color:white; text-shadow:0 2px 4px rgba(0,0,0,0.3); box-shadow:0 8px 32px rgba(99, 102, 241, 0.3);">🔑 Step 2: AES-256 Password Setup</h4>'),
                widgets.HTML('''
                <div style="background:linear-gradient(135deg, rgba(15, 23, 42, 0.95) 0%, rgba(30, 41, 59, 0.9) 100%);
                            border:2px solid rgba(99, 102, 241, 0.4); padding:20px; border-radius:16px;
                            margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.15); backdrop-filter:blur(10px);">
                    <div style="color:#00ff88; font-weight:bold; margin-bottom:12px; font-size:18px; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                        🔐 AES-256-GCM Security with zlib Compression:
                    </div>
                    <ul style="color:#e2e8f0; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                        <li style="margin:8px 0;">• <span style="color:#00d4ff;">256-bit encryption key</span> with quantum-resistant design</li>
                        <li style="margin:8px 0;">• <span style="color:#00ff88;">PBKDF2 with 100,000 iterations</span> for enhanced security</li>
                        <li style="margin:8px 0;">• <span style="color:#ffb000;">Cryptographically secure</span> salt & nonce generation</li>
                        <li style="margin:8px 0;">• <span style="color:#ff0040;">Built-in authentication</span> verification</li>
                        <li style="margin:8px 0;">• <span style="color:#a855f7; font-weight:bold;">NEW:</span> <span style="color:#00ff88;">Automatic zlib compression</span> (level 9)</li>
                    </ul>
                </div>
                ''')
            )

            # Update security section for dark mode
            self.security_section.children = (
                widgets.HTML('<h4 style="color:#e2e8f0; background:linear-gradient(135deg, rgba(15, 23, 42, 0.95) 0%, rgba(30, 41, 59, 0.9) 100%); padding:15px; border-radius:12px; box-shadow:0 8px 32px rgba(0,0,0,0.2); border:2px solid rgba(34, 197, 94, 0.4);">🛡️ Step 3: Security & Compression Options</h4>'),
                widgets.HTML('''
                <div style="background:linear-gradient(135deg, rgba(15, 23, 42, 0.95) 0%, rgba(30, 41, 59, 0.9) 100%);
                            border:2px solid rgba(34, 197, 94, 0.4); padding:20px; border-radius:16px;
                            margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.15); backdrop-filter:blur(10px);">
                    <div style="color:#00ff88; font-weight:bold; margin-bottom:12px; font-size:18px; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                        💡 Enhanced Security & Compression Features:
                    </div>
                    <ul style="color:#e2e8f0; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                        <li style="margin:8px 0;"><strong style="color:#ff0040;">SHA-256 Hash:</strong> <span style="color:#94a3b8;">Verifies message integrity during decoding</span></li>
                        <li style="margin:8px 0;"><strong style="color:#ff0040;">Timestamp:</strong> <span style="color:#94a3b8;">Records when the message was encoded</span></li>
                        <li style="margin:8px 0;"><strong style="color:#00d4ff;">zlib Compression:</strong> <span style="color:#94a3b8;">Automatically reduces data size when beneficial</span></li>
                        <li style="margin:8px 0;"><span style="color:#a855f7;">All features add minimal overhead but significantly improve efficiency</span></li>
                    </ul>
                </div>
                ''')
            )
        else:
            # Update password section for light mode
            self.password_section.children = (
                widgets.HTML('<h4 style="padding:15px; border-radius:12px; background:linear-gradient(135deg, #6366f1 0%, #8b5cf6 50%, #ec4899 100%); color:white; text-shadow:0 2px 4px rgba(0,0,0,0.3); box-shadow:0 8px 32px rgba(99, 102, 241, 0.3);">🔑 Step 2: AES-256 Password Setup</h4>'),
                widgets.HTML('''
                <div style="background:linear-gradient(135deg, rgba(248, 250, 252, 0.95) 0%, rgba(241, 245, 249, 0.9) 100%);
                            border:2px solid rgba(59, 130, 246, 0.4); padding:20px; border-radius:16px;
                            margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
                    <div style="color:#1e40af; font-weight:bold; margin-bottom:12px; font-size:18px;">
                        🔐 AES-256-GCM Security with zlib Compression:
                    </div>
                    <ul style="color:#374151; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                        <li style="margin:8px 0;">• <span style="color:#3b82f6;">256-bit encryption key</span> with quantum-resistant design</li>
                        <li style="margin:8px 0;">• <span style="color:#10b981;">PBKDF2 with 100,000 iterations</span> for enhanced security</li>
                        <li style="margin:8px 0;">• <span style="color:#f59e0b;">Cryptographically secure</span> salt & nonce generation</li>
                        <li style="margin:8px 0;">• <span style="color:#ef4444;">Built-in authentication</span> verification</li>
                        <li style="margin:8px 0;">• <span style="color:#8b5cf6; font-weight:bold;">NEW:</span> <span style="color:#10b981;">Automatic zlib compression</span> (level 9)</li>
                    </ul>
                </div>
                ''')
            )

            # Update security section for light mode
            self.security_section.children = (
                widgets.HTML('<h4 style="color:#1e293b; background:linear-gradient(135deg, rgba(248, 250, 252, 0.95) 0%, rgba(241, 245, 249, 0.9) 100%); padding:15px; border-radius:12px; box-shadow:0 8px 32px rgba(0,0,0,0.1); border:2px solid rgba(16, 185, 129, 0.4);">🛡️ Step 3: Security & Compression Options</h4>'),
                widgets.HTML('''
                <div style="background:linear-gradient(135deg, rgba(248, 250, 252, 0.95) 0%, rgba(241, 245, 249, 0.9) 100%);
                            border:2px solid rgba(16, 185, 129, 0.4); padding:20px; border-radius:16px;
                            margin:15px 0; box-shadow:0 20px 40px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
                    <div style="color:#059669; font-weight:bold; margin-bottom:12px; font-size:18px;">
                        💡 Enhanced Security & Compression Features:
                    </div>
                    <ul style="color:#374151; margin:0; padding-left:25px; line-height:1.8; font-size:14px;">
                        <li style="margin:8px 0;"><strong style="color:#dc2626;">SHA-256 Hash:</strong> <span style="color:#6b7280;">Verifies message integrity during decoding</span></li>
                        <li style="margin:8px 0;"><strong style="color:#dc2626;">Timestamp:</strong> <span style="color:#6b7280;">Records when the message was encoded</span></li>
                        <li style="margin:8px 0;"><strong style="color:#0284c7;">zlib Compression:</strong> <span style="color:#6b7280;">Automatically reduces data size when beneficial</span></li>
                        <li style="margin:8px 0;"><span style="color:#7c3aed;">All features add minimal overhead but significantly improve efficiency</span></li>
                    </ul>
                </div>
                ''')
            )

    def update_all_widgets(self):
        self.update_workflow_display()
        self.update_capacity_display()
        self.update_message_stats({'new': self.message_input.value})

    # ---------- Step 1: Image Upload ----------
    def handle_encode_upload(self, change):
        with self.encode_image_display:
            clear_output(wait=True)
            if change['new']:
                try:
                    uploaded = list(change['new'].values())[0]
                    img = Image.open(io.BytesIO(uploaded['content'])).convert('RGB')
                    self.original_image = np.array(img)

                    # Calculate capacity
                    h, w, c = self.original_image.shape
                    total_pixels = h * w * c
                    available_pixels = total_pixels - 64  # Reserve for header
                    self.max_capacity_bytes = available_pixels // 8

                    # Display preview with modern styling
                    display_img = img.resize((min(400, img.width), min(400, img.height)))
                    display(display_img)

                    self.workflow_step = 2
                    self.password_input.disabled = False
                    self.password_confirm.disabled = False

                    self.update_workflow_display()
                    self.update_capacity_display()
                    self.print_boxed_messages(["✅ Image uploaded successfully! You can now setup AES-256 password."], "success")

                except Exception as e:
                    self.print_boxed_messages([f"❌ Error loading image: {str(e)}"], "error")

    # ---------- Step 2: Password Validation ----------
    def validate_password(self, *_):
        """Enhanced password validation for AES-256"""
        colors = self.get_theme_colors()
        password = self.password_input.value
        confirm = self.password_confirm.value

        # Password strength analysis
        strength_score = 0
        feedback = []

        if len(password) >= 12:
            strength_score += 2
        elif len(password) >= 8:
            strength_score += 1
            feedback.append("Consider using 12+ characters")
        else:
            feedback.append("Password too short (min 8, recommended 12+)")

        if any(c.isupper() for c in password):
            strength_score += 1
        else:
            feedback.append("Add uppercase letters")

        if any(c.islower() for c in password):
            strength_score += 1
        else:
            feedback.append("Add lowercase letters")

        if any(c.isdigit() for c in password):
            strength_score += 1
        else:
            feedback.append("Add numbers")

        if any(c in "!@#$%^&*()_+-=[]{}|;:,.<>?" for c in password):
            strength_score += 1
        else:
            feedback.append("Add special characters")

        # Determine strength level with modern colors
        if strength_score >= 5:
            strength_level = "🔥 Excellent"
            strength_color = "linear-gradient(135deg, #10b981 0%, #34d399 100%)"
            strength_border = "#10b981"
        elif strength_score >= 4:
            strength_level = "💪 Strong"
            strength_color = "linear-gradient(135deg, #059669 0%, #10b981 100%)"
            strength_border = "#059669"
        elif strength_score >= 3:
            strength_level = "⚡ Good"
            strength_color = "linear-gradient(135deg, #f59e0b 0%, #fbbf24 100%)"
            strength_border = "#f59e0b"
        elif strength_score >= 2:
            strength_level = "⚠️ Weak"
            strength_color = "linear-gradient(135deg, #f97316 0%, #fb923c 100%)"
            strength_border = "#f97316"
        else:
            strength_level = "❌ Very Weak"
            strength_color = "linear-gradient(135deg, #dc2626 0%, #ef4444 100%)"
            strength_border = "#dc2626"

        # Password match validation
        passwords_match = password == confirm and len(password) >= 8
        match_status = "✅ Passwords match" if passwords_match else "❌ Passwords don't match" if len(confirm) > 0 else ""

        # Update strength display with modern design
        self.password_strength.value = f'''
        <div style="background:{colors['card_bg']}; border:2px solid {strength_border};
                    padding:15px; border-radius:12px; margin:15px 0; color:{colors['text_primary']};
                    box-shadow:0 10px 25px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
            <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:8px;">
                <div style="background:{strength_color}; padding:8px 15px; border-radius:20px; color:white; font-weight:bold; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                    {strength_level}
                </div>
                <span style="color:{colors['success'] if passwords_match else colors['error']}; font-weight:bold;">{match_status}</span>
            </div>
            {f'<div style="color:{colors["warning"]}; font-size:13px; margin-top:8px; padding:8px; background:rgba(245, 158, 11, 0.1); border-radius:8px;">💡 {", ".join(feedback)}</div>' if feedback else ''}
        </div>
        '''

        # Enable next step
        if passwords_match and len(password) >= 8:
            if self.workflow_step == 2:
                self.workflow_step = 3
                self.add_message_hash.disabled = False
                self.add_timestamp.disabled = False
                self.message_input.disabled = False
                self.update_workflow_display()
                self.print_boxed_messages(["✅ AES-256 password setup complete! You can now configure security options and enter your message."], "success")

    # ---------- Step 4: Message Input ----------
    def update_message_stats(self, change):
        """Update message statistics and capacity check with ACCURATE calculations including compression"""
        message = change['new']
        if not message:
            self.message_stats.value = ""
            return

        current_size = self.get_accurate_encrypted_size()
        effective_capacity = self.calculate_effective_capacity()

        # Test compression savings
        raw_size = len(message.encode('utf-8'))
        compressed_size = len(zlib.compress(message.encode('utf-8'), level=9))
        compression_savings = max(0, raw_size - compressed_size)
        compression_ratio = compressed_size / raw_size if raw_size > 0 else 1.0

        colors = self.get_theme_colors()

        if current_size > effective_capacity:
            status_gradient = "linear-gradient(135deg, #dc2626 0%, #ef4444 100%)"
            status_text = "❌ Too large!"
            progress_color = "#dc2626"
        elif current_size > effective_capacity * 0.8:
            status_gradient = "linear-gradient(135deg, #f97316 0%, #fb923c 100%)"
            status_text = "⚠️ Almost full"
            progress_color = "#f97316"
        else:
            status_gradient = "linear-gradient(135deg, #10b981 0%, #34d399 100%)"
            status_text = "✅ Fits perfectly"
            progress_color = "#10b981"

        # Compression status
        if compression_savings > 0:
            compression_status = f"🗜️ zlib saves {compression_savings} bytes ({(1-compression_ratio)*100:.1f}% reduction)"
            compression_gradient = "linear-gradient(135deg, #059669 0%, #10b981 100%)"
        else:
            compression_status = f"🗜️ No compression benefit (data not compressible)"
            compression_gradient = "linear-gradient(135deg, #3b82f6 0%, #60a5fa 100%)"

        self.message_stats.value = f'''
        <div style="background:{colors['card_bg']}; border:2px solid rgba(99, 102, 241, 0.3);
                    padding:20px; border-radius:16px; margin:15px 0; color:{colors['text_primary']};
                    box-shadow:0 20px 40px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
            <div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(200px, 1fr)); gap:15px; margin-bottom:15px;">
                <div style="padding:10px; border-radius:8px; background:rgba(59, 130, 246, 0.1); border:1px solid rgba(59, 130, 246, 0.3);">
                    <strong>Characters:</strong> {len(message)}
                </div>
                <div style="padding:10px; border-radius:8px; background:rgba(139, 92, 246, 0.1); border:1px solid rgba(139, 92, 246, 0.3);">
                    <strong>UTF-8 Bytes:</strong> {len(message.encode('utf-8'))}
                </div>
                <div style="padding:10px; border-radius:8px; background:rgba(236, 72, 153, 0.1); border:1px solid rgba(236, 72, 153, 0.3);">
                    <strong>Encrypted Size:</strong> {current_size} bytes
                </div>
                <div style="padding:10px; border-radius:8px; background:{status_gradient}; color:white; font-weight:bold; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                    {status_text}
                </div>
            </div>
            <div style="background:{compression_gradient}; color:white; margin:10px 0; padding:12px; border-radius:10px; font-size:13px; font-weight:bold; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                {compression_status}
            </div>
            <div style="background:{colors['bg_secondary']}; margin-top:15px; padding:12px; border-radius:10px; border:1px solid rgba(99, 102, 241, 0.2);">
                <div style="font-size:14px; margin-bottom:8px; font-weight:bold;">
                    Capacity Usage: {current_size:,}/{effective_capacity:,} bytes ({(current_size/effective_capacity*100):.1f}% used)
                </div>
                <div style="background:rgba(107, 114, 128, 0.2); height:8px; border-radius:4px; overflow:hidden;">
                    <div style="background:{progress_color}; height:100%; width:{min(100, current_size/effective_capacity*100):.1f}%;
                                border-radius:4px; transition:all 0.3s ease; box-shadow:0 0 10px {progress_color}50;"></div>
                </div>
            </div>
        </div>
        '''

        # Enable encode button
        if (len(message.strip()) > 0 and
            current_size <= effective_capacity and
            self.workflow_step >= 3):
            if self.workflow_step == 3 or self.workflow_step == 4:
                self.workflow_step = 5
                self.update_workflow_display()
            self.encode_button.disabled = False
        else:
            self.encode_button.disabled = True

    # ---------- Capacity Calculations (FIXED) ----------
    def update_capacity_display(self):
        """Update capacity display with accurate AES-256 + zlib overhead"""
        if self.original_image is None:
            return

        colors = self.get_theme_colors()
        effective_capacity = self.calculate_effective_capacity()
        h, w, c = self.original_image.shape

        self.capacity_display.value = f'''
        <div style="background:{colors['card_bg']}; border:2px solid rgba(16, 185, 129, 0.3);
                    padding:20px; border-radius:16px; margin:20px 0; color:{colors['text_primary']};
                    box-shadow:0 20px 40px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
            <h4 style="margin:0 0 15px 0; color:#10b981; font-size:20px; font-weight:bold; text-shadow:0 2px 4px rgba(0,0,0,0.1);">📊 Image Capacity Analysis</h4>
            <div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(250px, 1fr)); gap:15px; margin-bottom:15px;">
                <div style="padding:12px; border-radius:10px; background:linear-gradient(135deg, rgba(59, 130, 246, 0.1) 0%, rgba(99, 102, 241, 0.1) 100%); border:1px solid rgba(59, 130, 246, 0.3);">
                    <strong>Image dimensions:</strong> {h}×{w}×{c}
                </div>
                <div style="padding:12px; border-radius:10px; background:linear-gradient(135deg, rgba(139, 92, 246, 0.1) 0%, rgba(168, 85, 247, 0.1) 100%); border:1px solid rgba(139, 92, 246, 0.3);">
                    <strong>Total pixels:</strong> {h * w * c:,}
                </div>
                <div style="padding:12px; border-radius:10px; background:linear-gradient(135deg, rgba(236, 72, 153, 0.1) 0%, rgba(251, 113, 133, 0.1) 100%); border:1px solid rgba(236, 72, 153, 0.3);">
                    <strong>Available pixels:</strong> {h * w * c - 64:,}
                </div>
                <div style="padding:12px; border-radius:10px; background:linear-gradient(135deg, rgba(245, 158, 11, 0.1) 0%, rgba(251, 191, 36, 0.1) 100%); border:1px solid rgba(245, 158, 11, 0.3);">
                    <strong>Raw capacity:</strong> {self.max_capacity_bytes:,} bytes
                </div>
                <div style="padding:12px; border-radius:10px; background:linear-gradient(135deg, rgba(16, 185, 129, 0.1) 0%, rgba(52, 211, 153, 0.1) 100%); border:1px solid rgba(16, 185, 129, 0.3); grid-column:1/-1;">
                    <strong>Effective capacity (after AES-256 + zlib overhead):</strong> {effective_capacity:,} bytes
                </div>
            </div>
            <div style="background:linear-gradient(135deg, rgba(15, 23, 42, 0.9) 0%, rgba(30, 41, 59, 0.8) 100%);
                        padding:12px; border-radius:10px; margin-top:10px; font-size:13px; color:#94a3b8; border:1px solid rgba(99, 102, 241, 0.2);">
                💡 <strong style="color:#60a5fa;">AES-256-GCM + zlib Overhead includes:</strong> Salt (16B) + Nonce (12B) + Auth Tag (16B) + Base64 encoding + JSON structure + compression flag
            </div>
        </div>
        '''

    def calculate_effective_capacity(self):
        """Calculate capacity considering ACCURATE AES-256-GCM + zlib overhead"""
        if self.original_image is None:
            return 0

        base_capacity = self.max_capacity_bytes

        # Conservative overhead for JSON structure, base64 encoding, compression flag, etc.
        base_overhead = 220  # Slightly increased for compression flag

        return max(0, base_capacity - base_overhead)

    def get_accurate_encrypted_size(self):
        """Calculate ACCURATE encrypted size by simulating the actual process with compression"""
        if not self.message_input.value:
            return 0

        message = self.message_input.value

        # Create the exact payload that will be encrypted
        payload = {
            'msg': message,
            'ver': '2.2',  # Updated version
            'alg': 'AES-256-GCM'
        }

        if self.add_message_hash.value:
            payload['hash'] = hashlib.sha256(message.encode('utf-8')).hexdigest()

        if self.add_timestamp.value:
            payload['ts'] = datetime.now(timezone.utc).isoformat()

        # Convert to JSON bytes (exactly as in encoding)
        payload_json = json.dumps(payload, separators=(',', ':')).encode('utf-8')

        # Use the encryption class's size estimation method
        return self.encryption.estimate_encrypted_size(len(payload_json))

    # ---------- Step 5: Encoding ----------
    def encode_message(self, _):
        """Encode message with AES-256-GCM + zlib compression"""
        with self.encode_output:
            clear_output(wait=True)

            try:
                message = self.message_input.value.strip()
                password = self.password_input.value

                if not message:
                    self.print_boxed_messages(["❌ Please enter a message to encode"], "error")
                    return

                if not password or len(password) < 8:
                    self.print_boxed_messages(["❌ Please set a valid AES-256 password (minimum 8 characters)"], "error")
                    return

                # Create payload with metadata
                payload = {
                    'msg': message,
                    'ver': '2.2',  # Updated version for compression support
                    'alg': 'AES-256-GCM'
                }

                if self.add_message_hash.value:
                    payload['hash'] = hashlib.sha256(message.encode('utf-8')).hexdigest()
                    self.print_boxed_messages(["🔐 Adding SHA-256 message hash for integrity verification"], "info")

                if self.add_timestamp.value:
                    payload['ts'] = datetime.now(timezone.utc).isoformat()
                    self.print_boxed_messages(["🕒 Adding timestamp metadata"], "info")

                # Convert to JSON
                payload_json = json.dumps(payload, separators=(',', ':')).encode('utf-8')
                self.print_boxed_messages([f"📊 Payload size: {len(payload_json)} bytes"], "info")

                # Test compression
                compressed_test = zlib.compress(payload_json, level=9)
                compression_savings = len(payload_json) - len(compressed_test)
                if compression_savings > 0:
                    self.print_boxed_messages([f"🗜️ zlib compression will save {compression_savings} bytes ({(compression_savings/len(payload_json)*100):.1f}% reduction)"], "success")
                else:
                    self.print_boxed_messages([f"🗜️ zlib compression not beneficial for this data"], "info")

                # Encrypt with AES-256-GCM + zlib
                self.print_boxed_messages(["🔐 Encrypting with AES-256-GCM + zlib compression..."], "info")
                encrypted_data = self.encryption.encrypt_aes256_gcm(payload_json, password)

                # Verify capacity
                if len(encrypted_data) > self.calculate_effective_capacity():
                    self.print_boxed_messages([f"❌ Encrypted data ({len(encrypted_data)} bytes) exceeds image capacity ({self.calculate_effective_capacity()} bytes)"], "error")
                    return

                # Generate dynamic shuffle ID
                shuffle_id = int(hashlib.sha256(password.encode()).hexdigest()[:16], 16)
                self.print_boxed_messages([f"🔀 Generated shuffle ID: {shuffle_id:016x}"], "info")

                # Encode into image
                self.print_boxed_messages(["📷 Embedding data into image with dynamic pixel shuffling..."], "info")
                encoded_image = self.embed_data_dynamic(self.original_image.copy(), encrypted_data, shuffle_id)

                if encoded_image is not None:
                    self.encoded_result = encoded_image
                    self.download_button.layout.display = 'block'
                    self.workflow_step = 6
                    self.update_workflow_display()

                    # Display result preview
                    result_pil = Image.fromarray(encoded_image)
                    display_img = result_pil.resize((min(400, result_pil.width), min(400, result_pil.height)))
                    display(display_img)

                    # Display success messages in a box
                    success_messages = [
                        "✅ Message encoded successfully with AES-256-GCM + zlib!",
                        f"🔐 Final encrypted size: {len(encrypted_data)} bytes",
                        f"📊 Capacity usage: {len(encrypted_data)/self.calculate_effective_capacity()*100:.1f}%",
                        "📥 Click 'Download Encoded Image' to save the result"
                    ]
                    self.print_boxed_messages(success_messages, "success")
                else:
                    self.print_boxed_messages(["❌ Failed to encode data into image"], "error")

            except Exception as e:
                self.print_boxed_messages([f"❌ Encoding failed: {str(e)}"], "error")

    def embed_data_dynamic(self, image, data, shuffle_id):
        """Embed data using dynamic pixel shuffling"""
        try:
            h, w, c = image.shape
            flat_image = image.flatten()

            # Convert data to bits
            data_bits = ''.join(format(byte, '08b') for byte in data)

            # Add header (64 bits for data length)
            header = format(len(data_bits), '064b')
            full_bits = header + data_bits

            if len(full_bits) > len(flat_image) - 64:
                raise ValueError("Data too large for image capacity")

            # Generate shuffle sequence
            shuffle_seq = self.shuffle_engine.generate_shuffle_sequence(len(flat_image) - 64, shuffle_id)

            # Embed header in first 64 pixels
            for i in range(64):
                flat_image[i] = (flat_image[i] & 0xFE) | int(header[i])

            # Embed data using shuffled positions
            for i, bit in enumerate(data_bits):
                pos = shuffle_seq[i] + 64  # Offset by header size
                flat_image[pos] = (flat_image[pos] & 0xFE) | int(bit)

            return flat_image.reshape(h, w, c)

        except Exception as e:
            print(f"Embedding error: {e}")
            return None

    # ---------- Decoding ----------
    def handle_decode_upload(self, change):
        """Handle decode image upload"""
        with self.decode_image_display:
            clear_output(wait=True)
            if change['new']:
                try:
                    uploaded = list(change['new'].values())[0]
                    img = Image.open(io.BytesIO(uploaded['content'])).convert('RGB')
                    self.decode_image = np.array(img)

                    # Display preview
                    display_img = img.resize((min(400, img.width), min(400, img.height)))
                    display(display_img)

                    self.print_boxed_messages(["✅ Encoded image loaded successfully!"], "success")

                except Exception as e:
                    self.print_boxed_messages([f"❌ Error loading image: {str(e)}"], "error")

    def decode_message(self, _):
        """Decode message with AES-256-GCM + zlib decompression"""
        with self.decode_output:
            clear_output(wait=True)

            try:
                if self.decode_image is None:
                    self.print_boxed_messages(["❌ Please upload an encoded image first"], "error")
                    return

                password = self.decode_password.value
                if not password:
                    self.print_boxed_messages(["❌ Please enter the AES-256 password"], "error")
                    return

                # Generate shuffle ID from password
                shuffle_id = int(hashlib.sha256(password.encode()).hexdigest()[:16], 16)
                self.print_boxed_messages([f"🔀 Using shuffle ID: {shuffle_id:016x}"], "info")

                # Extract data
                self.print_boxed_messages(["📷 Extracting data from image..."], "info")
                extracted_data = self.extract_data_dynamic(self.decode_image, shuffle_id)

                if extracted_data is None:
                    self.print_boxed_messages(["❌ No valid data found in image"], "error")
                    return

                # Decrypt with AES-256-GCM + zlib
                self.print_boxed_messages(["🔓 Decrypting with AES-256-GCM + zlib decompression..."], "info")
                decrypted_data = self.encryption.decrypt_aes256_gcm(extracted_data, password)

                # Parse JSON payload
                payload = json.loads(decrypted_data.decode('utf-8'))
                message = payload['msg']

                # Verify integrity if hash exists
                if 'hash' in payload:
                    expected_hash = payload['hash']
                    actual_hash = hashlib.sha256(message.encode('utf-8')).hexdigest()
                    if expected_hash == actual_hash:
                        self.print_boxed_messages(["✅ SHA-256 hash verification passed - message integrity confirmed"], "success")
                    else:
                        self.print_boxed_messages(["❌ SHA-256 hash verification failed - message may be corrupted"], "error")
                        return

                # Display timestamp if exists
                if 'ts' in payload:
                    timestamp = payload['ts']
                    self.print_boxed_messages([f"🕒 Message encoded at: {timestamp}"], "info")

                # Display algorithm info
                algorithm = payload.get('alg', 'Unknown')
                version = payload.get('ver', '1.0')
                self.print_boxed_messages([f"🔐 Encryption: {algorithm} (version {version})"], "info")

                # Display decoded messages in a box
                decoded_messages = [
                    "✅ Message decoded successfully!",
                    f"📊 Message length: {len(message)} characters",
                    f"📄 UTF-8 size: {len(message.encode('utf-8'))} bytes"
                ]
                self.print_boxed_messages(decoded_messages, "success")

                # Display the message in a styled box with FIXED colors for dark mode
                colors = self.get_theme_colors()
                message_text_color = "#e5e7eb" if self.dark_mode else "#1e293b"  # Fixed: Light gray text for dark mode
                message_bg_color = "#374151" if self.dark_mode else "#f8fafc"    # Fixed: Dark gray background for dark mode

                message_display = f'''
                <div style="background:{colors['card_bg']}; border:2px solid rgba(16, 185, 129, 0.4);
                            padding:20px; border-radius:16px; margin:20px 0;
                            box-shadow:0 20px 40px rgba(0,0,0,0.1); backdrop-filter:blur(10px);">
                    <h4 style="margin:0 0 15px 0; color:#10b981; font-size:18px; font-weight:bold;">📝 Decoded Message:</h4>
                    <div style="background:{message_bg_color}; padding:15px; border-radius:10px;
                                font-family:monospace; font-size:14px; line-height:1.6; word-wrap:break-word;
                                border:1px solid rgba(99, 102, 241, 0.2); max-height:300px; overflow-y:auto;
                                color:{message_text_color};">
                        {message.replace('<', '&lt;').replace('>', '&gt;')}
                    </div>
                </div>
                '''
                display(HTML(message_display))

            except json.JSONDecodeError:
                self.print_boxed_messages(["❌ Invalid data format - not a valid encoded message"], "error")
            except Exception as e:
                self.print_boxed_messages([f"❌ Decoding failed: {str(e)}"], "error")

    def extract_data_dynamic(self, image, shuffle_id):
        """Extract data using dynamic pixel shuffling"""
        try:
            h, w, c = image.shape
            flat_image = image.flatten()

            # Extract header (64 bits)
            header_bits = ''.join(str(flat_image[i] & 1) for i in range(64))
            data_length = int(header_bits, 2)

            if data_length <= 0 or data_length > (len(flat_image) - 64):
                return None

            # Generate same shuffle sequence
            shuffle_seq = self.shuffle_engine.generate_shuffle_sequence(len(flat_image) - 64, shuffle_id)

            # Extract data bits using shuffled positions
            data_bits = []
            for i in range(data_length):
                pos = shuffle_seq[i] + 64
                data_bits.append(str(flat_image[pos] & 1))

            # Convert bits to bytes
            data_bytes = bytearray()
            for i in range(0, len(data_bits), 8):
                if i + 8 <= len(data_bits):
                    byte_bits = ''.join(data_bits[i:i+8])
                    data_bytes.append(int(byte_bits, 2))

            return bytes(data_bytes)

        except Exception as e:
            print(f"Extraction error: {e}")
            return None

    def download_encoded_image(self, _):
        """Handle download of encoded image"""
        if self.encoded_result is not None:
            try:
                # Convert to PIL Image
                result_pil = Image.fromarray(self.encoded_result)

                # Create filename with timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"bengali_stego_encoded_{timestamp}.png"

                # Save file first, then download
                result_pil.save(filename, format='PNG', optimize=True)

                # Now download the saved file
                files.download(filename)

                self.print_boxed_messages([f"✅ Image saved and downloaded as {filename}"], "success")

            except Exception as e:
                # Fallback: Try base64 display method
                try:
                    result_pil = Image.fromarray(self.encoded_result)
                    img_buffer = io.BytesIO()
                    result_pil.save(img_buffer, format='PNG', optimize=True)
                    img_buffer.seek(0)

                    # Create base64 download link
                    img_data = base64.b64encode(img_buffer.getvalue()).decode()
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"bengali_stego_encoded_{timestamp}.png"

                    download_html = f'''
                    <div style="text-align:center; padding:20px; background:linear-gradient(135deg, #10b981 0%, #34d399 100%);
                                border-radius:15px; margin:15px 0; box-shadow:0 10px 25px rgba(16, 185, 129, 0.3);">
                        <h4 style="color:white; margin:0 0 15px 0; text-shadow:0 2px 4px rgba(0,0,0,0.3);">📥 Download Your Encoded Image</h4>
                        <a href="data:image/png;base64,{img_data}"
                           download="{filename}"
                           style="display:inline-block; padding:12px 24px; background:rgba(255,255,255,0.2);
                                  color:white; text-decoration:none; border-radius:10px; font-weight:bold;
                                  border:2px solid rgba(255,255,255,0.3); backdrop-filter:blur(10px);
                                  transition:all 0.3s ease; text-shadow:0 2px 4px rgba(0,0,0,0.3);">
                            🔐 Download {filename}
                        </a>
                        <p style="color:rgba(255,255,255,0.9); margin:10px 0 0 0; font-size:14px;">
                            Right-click the link above and select "Save As" if automatic download doesn't work
                        </p>
                    </div>
                    '''
                    display(HTML(download_html))
                    self.print_boxed_messages([f"✅ Download link created for {filename}"], "success")

                except Exception as fallback_error:
                    self.print_boxed_messages([f"❌ Download failed: {str(e)}. Fallback error: {str(fallback_error)}"], "error")

    # ---------- Main Interface ----------
    def display(self):
        """Display the main interface"""
        display(self.main_container)

# ---------- Usage ----------
def create_bengali_steganography_suite():
    """Create and display the Bengali Steganography Suite"""
    stego = BengaliSteganographyAES256()
    stego.display()
    return stego

# Initialize the application
if __name__ == "__main__":
    app = create_bengali_steganography_suite()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>